## Laboratorio 4 - Algoritmos Genéticos (Convocatoria Extraordinaria)
### Inteligencia Artificial II

### Miguel Márquez Gonzalez

---

### Librerías usadas

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random as rd
from scipy import stats

---

### Variables y parámetros

In [40]:
NTOWN = 12
NPOB = 20
NGEN = 2000
Q = 0.9
NRES = 50
NSAMPLE = 200

Diccionario para mapear letras de ciudades con sus ids

In [3]:
def lett_to_id(l):
    
    l_i = {
        'CAM': 0, 
        'CAN': 1,
        'CHE': 2,
        'CHI': 3,
        'IZ':  4,
        'PC':  5,
        'PM':  6,
        'ME':  7,
        'TI':  8,
        'TU':  9,
        'UX':  10,
        'VA':  11
    }
    
    return l_i[l]

Diccionario para mapear ids con las letras de ciudades

In [4]:
def id_to_lett(i):
    
    i_l = {
        0: 'CAM', 
        1: 'CAN',
        2: 'CHE',
        3: 'CHI',
        4: 'IZ',
        5: 'PC',
        6: 'PM',
        7: 'ME',
        8: 'TI',
        9: 'TU',
        10:'UX',
        11:'VA'
    }
    
    return i_l[i]

---

### Funciones implementadas

Generacion de aletarios mejor que la funcion usual de Python. Genera un valor entre b y t (bottom, top).

In [5]:
def aleatorio(b, t):
    random_data = os.urandom(8)

    #Coge la sucesión de carácteres criptográficos obtenidos y los transformamos a un int, actua como semilla
    seed = int.from_bytes(random_data, byteorder="big") 
    rd.seed(seed)
    
    return (rd.randint(b,t)/t)

Matriz para almacenar y devolver la distancias entre ciudades

In [6]:
def dist(c1, c2):
    
    id1 = lett_to_id(c1)
    id2 = lett_to_id(c2)
    
    distancias = np.array([[  0, 515, 425, 315, 265, 583, 551, 196, 409, 470, 171, 355],
                           [515,   0, 382, 205, 266,  68,  36, 320, 212, 131, 399, 160],
                           [425, 382,   0, 410, 471, 314, 346, 525, 417, 251, 605, 365],
                           [315, 205, 410,   0,  62, 273, 241, 115,  97, 159, 199,  41],
                           [265, 266, 471,  62,   0, 334, 302,  72, 159, 220, 149, 106],
                           [583,  68, 314, 273, 334,   0,  32, 308, 279,  63, 468, 228],
                           [551,  36, 346, 241, 302,  32,   0, 256, 297,  95, 435, 195],
                           [196, 320, 525, 115,  72, 308, 256,   0, 212, 274,  80, 160],
                           [409, 212, 417,  97, 159, 279, 297, 212,   0, 166, 291,  52],
                           [470, 131, 251, 159, 220,  63,  95, 274, 166,   0, 366, 291],
                           [171, 399, 605, 199, 149, 468, 435,  80, 291, 366,   0, 239],
                           [355, 169, 265,  41, 106, 228, 195, 160,  52, 291, 239, 0]])
    
    return distancias[id1, id2]

Generar un individuo aleatorio

In [7]:
def nuevo_individuo(NTOWN):
    
    origen = int(aleatorio(0, NTOWN-1)*(NTOWN-1))
    
    ruta = list(range(NTOWN))
    del ruta[origen]    
    rd.shuffle(ruta)
    ruta.append(origen)
    ruta.insert(0, origen)
        
    individuo = indiv_to_lett(ruta)
    
    return individuo

Generar población aleatoria

In [8]:
def nueva_poblacion(NTOWN, NPOB):
    
    poblacion = []
    
    for i in range(NPOB):
        poblacion.append(nuevo_individuo(NTOWN))
    
    return poblacion

Convierte individuo de ids a iniciales

In [9]:
def indiv_to_lett(i):
    n_i = []
    for g in i:
        n_i.append(id_to_lett(g))
    return n_i

Convierte individuo de iniciales a ids

In [10]:
def indiv_to_ids(i):
    n_i = []
    for g in i:
        n_i.append(lett_to_id(g))
    return n_i

Calcular Fitness de una poblacion

In [11]:
def fitness_p(p):
    
    fitness_pob = []
    
    mi = 999999
    ma = 0
    
    for i in p:
        t = fitness_i(i)
        if t > ma: ma = t
        if t < mi: mi = t
        fitness_pob.append(t)
    
    me = np.sum(fitness_pob)/len(fitness_pob)
        
    return fitness_pob, mi, me, ma

Calcular Fitness de un individuo

In [12]:
def fitness_i(i):
    
    coste_total = 0
    
    for c in range(len(i)-1):
        coste_total += dist(i[c], i[c+1])
    
    return coste_total

Calcular la probabilidad de que un individuo se replique

In [13]:
def replicacion(i, fitness):
    
    # Probabilidad de seleccion de ese individuo
    psi = 1 - ((fitness[i]- np.min(fitness))/np.max(fitness))

    # Probabilidad por el Método de Montecarlo simple
    pmm = aleatorio(0, 1000)
    
    return (psi >= pmm)

Mutación de un individuo

In [14]:
def mutar(individuo, Q):
    
    nuevo_individuo = individuo.copy()
    n = len(nuevo_individuo)-2 # Para evitar la ultima ciudad
    
    # Probabilidad por el Método de Montecarlo simple
    pmm = aleatorio(0, 1000)
    
    # Q es la prob de NO mutar
    if (Q < pmm):
    
        c1 = int(aleatorio(1,n)*n)
        c2 = int(aleatorio(1,n)*n)
        while(c1 == c2): c2 = int(aleatorio(1,n)*n)
        
        # Cambio de orden
        nuevo_individuo[c2] = individuo[c1]
        nuevo_individuo[c1] = individuo[c2]
        
#         print('MUTA', individuo, t1, '<->', t2, nuevo_individuo)
    
    return nuevo_individuo

Imprimir una poblacion de forma visualmente placentera

In [15]:
def print_pob(poblacion, p):
    
    n = int(len(poblacion)*p)
    
    for i in poblacion[:n]:
        print(i)

Calcular cual es el mejor individuo y cuantas veces aparece

In [16]:
def veces_mejor(poblacion):
    
    cont = 0
    mejor = poblacion[np.argmin(fitness)]
    
    for i in poblacion:
        if i == mejor:
            cont += 1
    
    veces = (cont/len(poblacion))*100
    
    return mejor, veces

---

### Generar primera población

In [41]:
poblacion = nueva_poblacion(NTOWN, NPOB)
fitness, _, _, _ = fitness_p(poblacion)
print_pob(poblacion, 1)

['CHE', 'CHI', 'CAN', 'TU', 'CAM', 'UX', 'IZ', 'PM', 'VA', 'ME', 'TI', 'PC', 'CHE']
['TI', 'UX', 'CAN', 'VA', 'ME', 'IZ', 'PC', 'CHE', 'PM', 'TU', 'CHI', 'CAM', 'TI']
['VA', 'CHI', 'PM', 'TU', 'IZ', 'UX', 'CHE', 'CAN', 'ME', 'TI', 'PC', 'CAM', 'VA']
['CHE', 'IZ', 'ME', 'VA', 'TI', 'CAM', 'CHI', 'TU', 'UX', 'PC', 'CAN', 'PM', 'CHE']
['ME', 'TI', 'PM', 'PC', 'CHE', 'IZ', 'CAN', 'CAM', 'UX', 'VA', 'CHI', 'TU', 'ME']
['PC', 'UX', 'IZ', 'CAM', 'PM', 'TU', 'CAN', 'CHE', 'CHI', 'ME', 'VA', 'TI', 'PC']
['UX', 'IZ', 'PC', 'CAM', 'CHE', 'TI', 'ME', 'CHI', 'CAN', 'VA', 'PM', 'TU', 'UX']
['IZ', 'CAM', 'ME', 'TU', 'UX', 'VA', 'CAN', 'TI', 'PM', 'CHI', 'PC', 'CHE', 'IZ']
['PC', 'VA', 'TI', 'IZ', 'CHE', 'CHI', 'PM', 'ME', 'UX', 'CAM', 'TU', 'CAN', 'PC']
['ME', 'PM', 'IZ', 'VA', 'CAM', 'UX', 'TI', 'TU', 'PC', 'CHE', 'CHI', 'CAN', 'ME']
['CHI', 'PC', 'CHE', 'UX', 'CAN', 'TU', 'ME', 'CAM', 'TI', 'PM', 'IZ', 'VA', 'CHI']
['PC', 'PM', 'UX', 'CHI', 'IZ', 'ME', 'CHE', 'CAM', 'TU', 'TI', 'CAN', 'VA', 'PC']
[

### Bucle del AG

In [42]:
for gen in range(NGEN):

    # Se copia la poblacion para hacer en ella los cambios
    n_poblacion = poblacion.copy()
    
    # Individuo inicial
    i = rd.randint(0,NPOB-1)
    
    # Seleccionar ultimo individuo
    if (i==0):
        fin = NPOB-1 
    else:
        fin = i-1
        
    # Pasar por cada individuo
    while (not i == fin):
        
        # Decidir si el padre se repetirá en la nueva población
        if (replicacion(i, fitness)):
            n_poblacion[int(aleatorio(0,NPOB-1)*(NPOB-1))] = mutar(poblacion[i], Q)

        if (i == NPOB-1):
            i=0
        else:
            i +=1
            
    # Calculo del nuevo fitness
    poblacion = n_poblacion.copy()
    fitness, mi, me, ma = fitness_p(poblacion)
    
    # Imrpimir informacion
    if gen % NRES == 0:
        
        mejor, veces = veces_mejor(poblacion)
        consenso = stats.mode(poblacion).mode
        
        print('\n********************************************************************************************************')
        print('Generación:', gen)
        print('Distancias:', 'Mínima', mi, 'Media', me, 'Máxima', ma)
        print('Mejor individuo:', mejor)
        print('Aparecece en el', veces, '% de indiviuos')
        print('Individuo consenso:', consenso)
        
        
        
    if gen % NSAMPLE == 0:
        print('\n20% de la población:')
        print_pob(poblacion, 0.2)



********************************************************************************************************
Generación: 0
Distancias: Mínima 2737 Media 3097.85 Máxima 3536
Mejor individuo: ['PC', 'VA', 'TI', 'IZ', 'CHE', 'CHI', 'PM', 'ME', 'UX', 'CAM', 'TU', 'CAN', 'PC']
Aparecece en el 10.0 % de indiviuos
Individuo consenso: [['PC' 'PM' 'IZ' 'VA' 'CHE' 'TU' 'PM' 'CAM' 'UX' 'VA' 'CHI' 'CAN' 'PC']]

20% de la población:
['CHE', 'CHI', 'CAN', 'TU', 'CAM', 'UX', 'IZ', 'PM', 'VA', 'ME', 'TI', 'PC', 'CHE']
['TI', 'UX', 'CAN', 'VA', 'ME', 'IZ', 'PC', 'CHE', 'PM', 'TU', 'CHI', 'CAM', 'TI']
['TI', 'UX', 'CAN', 'VA', 'ME', 'IZ', 'PC', 'CHE', 'PM', 'TU', 'CHI', 'CAM', 'TI']
['CHI', 'PC', 'CHE', 'UX', 'CAN', 'TU', 'ME', 'CAM', 'TI', 'PM', 'IZ', 'VA', 'CHI']

********************************************************************************************************
Generación: 50
Distancias: Mínima 2549 Media 2982.0 Máxima 3478
Mejor individuo: ['UX', 'CHI', 'IZ', 'CAM', 'ME', 'TI', 'CHE', 'PC', 'CAN',


********************************************************************************************************
Generación: 850
Distancias: Mínima 2297 Media 2484.55 Máxima 2818
Mejor individuo: ['UX', 'CAM', 'IZ', 'TI', 'CHI', 'TU', 'PM', 'VA', 'CAN', 'PC', 'CHE', 'ME', 'UX']
Aparecece en el 10.0 % de indiviuos
Individuo consenso: [['UX' 'CAM' 'CHI' 'TI' 'IZ' 'TU' 'PM' 'VA' 'CAN' 'PC' 'CHE' 'ME' 'UX']]

********************************************************************************************************
Generación: 900
Distancias: Mínima 2238 Media 2395.3 Máxima 2837
Mejor individuo: ['UX', 'CAM', 'IZ', 'TI', 'CHI', 'VA', 'PM', 'TU', 'CAN', 'CHE', 'PC', 'ME', 'UX']
Aparecece en el 55.00000000000001 % de indiviuos
Individuo consenso: [['UX' 'CAM' 'IZ' 'TI' 'CHI' 'VA' 'PM' 'TU' 'CAN' 'CHE' 'PC' 'ME' 'UX']]

********************************************************************************************************
Generación: 950
Distancias: Mínima 2014 Media 2308.8 Máxima 2706
Mejor individuo


********************************************************************************************************
Generación: 1700
Distancias: Mínima 1777 Media 1996.0 Máxima 2396
Mejor individuo: ['UX', 'ME', 'VA', 'CHI', 'IZ', 'TI', 'TU', 'PM', 'CAN', 'PC', 'CHE', 'CAM', 'UX']
Aparecece en el 5.0 % de indiviuos
Individuo consenso: [['UX' 'ME' 'VA' 'CHI' 'TI' 'IZ' 'TU' 'PM' 'CAN' 'PC' 'CHE' 'CAM' 'UX']]

********************************************************************************************************
Generación: 1750
Distancias: Mínima 1866 Media 2079.35 Máxima 2658
Mejor individuo: ['UX', 'ME', 'VA', 'CHI', 'TI', 'IZ', 'TU', 'PM', 'CAN', 'PC', 'CHE', 'CAM', 'UX']
Aparecece en el 55.00000000000001 % de indiviuos
Individuo consenso: [['UX' 'ME' 'VA' 'CHI' 'TI' 'IZ' 'TU' 'PM' 'CAN' 'PC' 'CHE' 'CAM' 'UX']]

********************************************************************************************************
Generación: 1800
Distancias: Mínima 1777 Media 1970.85 Máxima 2905
Mejor indivi

---